In [1]:
import sys, os, glob, pickle, toml, json
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import cartopy.crs as ccrs
import cartopy.feature as cfeature

def set_background(ax):
    ax.set_extent([-121, -72, 25, 50])

    # Background boundaries
    # ax.add_feature(cfeature.LAND)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS)
    states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')
    ax.add_feature(states_provinces, edgecolor='gray')
    
    return ax


def cal_ratio_within_bins(data, bins):
    binsc = np.zeros(len(bins)-1)
    ratio = np.zeros(len(bins)-1)
    for i in range(len(bins)-1):
        indi = (data>bins[i]) & (data<=bins[i+1])
        ratio[i] = np.sum(indi)/np.sum(~np.isnan(data))
        binsc[i] = (bins[i]+bins[i+1])/2
    return binsc, ratio

In [2]:
# load info
infile_basin_info = f"/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv"
df_info = pd.read_csv(infile_basin_info)

In [5]:
# input/out and iteration information
iternum = 2
inpath_moasmo = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO_bigrange/'

In [6]:
# LSE metrics
# outfile = f'dtmp.gz'
# if os.path.isfile(outfile):
if False:
    df_lse_metric = pd.read_csv(outfile, compression='gzip')
else:
    df_lse_metric = []

    for tarbasin in range(len(df_info)):
        if not tarbasin in np.arange(0, 627, 5):
            for i in range(iternum):
                infile = f'{inpath_moasmo}/level1_{tarbasin}_MOASMOcalib/ctsm_outputs_LSEtrain/iter{i}_many_metric.csv'    
                dfi = pd.read_csv(infile)
                dfi['basin'] = tarbasin
                dfi['iter'] = i
                dfi['trial'] = np.arange(len(dfi))
                if len(df_lse_metric) == 0:
                    df_lse_metric = dfi
                else:
                    df_lse_metric = pd.concat([df_lse_metric, dfi])

In [9]:
metname = 'kge'
trainbasin = np.unique(df_lse_metric['basin'].values)
met_stats_lse = np.nan * np.zeros([len(trainbasin), iternum, 3]) # best; mean; median
for i in range(iternum):
    indi = (df_lse_metric['iter']==i)
    for j in range(len(trainbasin)):
        indij = indi & (df_lse_metric['basin']==trainbasin[j])
        dij = df_lse_metric[indij][metname].values

        met_stats_lse[j, i, 0] = np.nanmax(dij)
        met_stats_lse[j, i, 1] = np.nanmean(dij)
        met_stats_lse[j, i, 2] = np.nanmedian(dij)



In [11]:
# iter-1 still improves against iter-0
np.nanmean(met_stats_lse[:,:,0],axis=0)

array([0.37585942, 0.42626141])